In [35]:
# %% [Section 1] 라이브러리 임포트
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# --------------------------------------------------------------------------------
# [Part 1] 기존 FFFFFFFINAL_ENS.ipynb의 핵심 전처리 함수들
# --------------------------------------------------------------------------------

# 1. 기본 전처리 (나이 변환 등)
def prepare_raw_simple(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "birth_date" in df.columns:
        df["birth_date"] = pd.to_datetime(df["birth_date"])
        # 기준일은 2025년으로 가정 (기존 코드 유지)
        df["age"] = 2025 - df["birth_date"].dt.year
    
    # gender 숫자 변환 (XGB용, CatBoost용은 원본 gender를 쓰지만 여기선 미리 만들어둠)
    if "gender" in df.columns:
        df["gender_int"] = df["gender"].map({"M": 0, "F": 1})
    return df

# 2. EDA Rule Features
def add_eda_rule_features_v2(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) chem_01_range
    df['chem_01_range_str'] = pd.cut(
        df['chem_01'], 
        bins=[-np.inf, 2.0, 4.0, np.inf], 
        labels=['Normal_Low', 'Warning_Zone', 'Danger_High']
    ).astype(str)

    chem_map = {'Normal_Low': 0, 'Warning_Zone': 1, 'Danger_High': 2}
    df['chem_01_range'] = df['chem_01_range_str'].map(chem_map).astype(int)

    # 2) trace_metal_range
    df['trace_metal_range_str'] = pd.cut(
        df['trace_metal'],
        bins=[-np.inf, 90, 130, np.inf],
        labels=['Safe', 'Caution', 'Risk']
    ).astype(str)

    metal_map = {'Safe': 0, 'Caution': 1, 'Risk': 2}
    df['trace_metal_range'] = df['trace_metal_range_str'].map(metal_map).astype(int)

    # 3) risk_segment
    def check_ambiguous_risk(row):
        if (row['swelling'] == 'S') and (2.0 <= row['chem_01'] < 5.0):
            return 'Target_1_Suspect'
        elif (row['swelling'] == 'Y') or (row['chem_01'] >= 5.0):
            return 'High_Risk'
        else:
            return 'Normal'

    df['risk_segment_str'] = df.apply(check_ambiguous_risk, axis=1).astype(str)
    risk_map = {'Normal': 0, 'Target_1_Suspect': 1, 'High_Risk': 2}
    df['risk_segment'] = df['risk_segment_str'].map(risk_map).astype(int)

    return df

# 3. Boosting Features
def make_boosting_features_v2(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 기본 수치형 파생
    high_is_healthy = ["protein_level", "blood_cells"]
    low_is_healthy  = ["enzyme_A", "enzyme_B", "lipid_index", "trace_metal", "clot_time"]

    def minmax(s):
        s = s.astype(float)
        mn, mx = s.min(), s.max()
        if mx == mn: return pd.Series(0.5, index=s.index)
        return (s - mn) / (mx - mn + 1e-6)

    if all(col in df.columns for col in high_is_healthy + low_is_healthy):
        df["health_index"] = (
            sum(minmax(df[col]) for col in high_is_healthy)
            - sum(minmax(df[col]) for col in low_is_healthy)
        )
        df["health_per_stage"] = df["health_index"] / (df["disease_stage"] + 1e-6)

    # 상호작용 / 비율 / 합·차
    df["chem01_trace_combo"]   = df["chem_01"] * df["trace_metal"]
    df["chem01_chem02_combo"]  = df["chem_01"] * df["chem_02"]
    df["chem01_enzymeB_combo"] = df["chem_01"] * df["enzyme_B"]
    df["enzyme_ratio"]       = df["enzyme_A"] / (df["enzyme_B"].replace(0, np.nan) + 1e-6)
    df["lipid_blood_ratio"]  = df["lipid_index"] / (df["blood_cells"].replace(0, np.nan) + 1e-6)
    df["chem02_trace_ratio"] = df["chem_02"] / (df["trace_metal"].replace(0, np.nan) + 1e-6)
    
    df["sum_chem"]   = df["chem_01"] + df["chem_02"]
    df["diff_chem"]  = df["chem_01"] - df["chem_02"]
    df["sum_enzyme"] = df["enzyme_A"] + df["enzyme_B"]
    df["diff_enzyme"] = df["enzyme_A"] - df["enzyme_B"]

    # 시간/나이 정규화
    if "age" in df.columns:
        df["obs_per_age"]      = df["obs_days"] / (df["age"] + 1e-6)
        df["behavior_per_age"] = df["behavior_index"] / (df["age"] + 1e-6)
    df["disease_velocity"] = df["disease_stage"] / (df["obs_days"] + 1e-6)

    # 증상 카운트 & Swelling 변환
    symptom_cols = ["fluid_accum", "organ_enlarge", "vascular_marks"]
    # replace 시 안전하게 처리
    temp_symptoms = df[symptom_cols].replace({"Y": 1, "N": 0}).infer_objects(copy=False)
    df["symptom_count"] = temp_symptoms.sum(axis=1)

    if "swelling" in df.columns:
        df["swelling_ord"] = df["swelling"].map({"N": 0, "S": 1, "Y": 2})
    else:
        df["swelling_ord"] = 0
    df["symptom_severity"] = df["symptom_count"] + df["swelling_ord"]
    
    return df

# --------------------------------------------------------------------------------
# [Part 2] 🔥 NEW: Mayo Score & Risk Group (새로 추가된 핵심 피처)
# --------------------------------------------------------------------------------

def add_mayo_pbc_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # 1. 나이(Age) 계산 (이미 prepare_raw_simple에서 계산됨, 없으면 안전장치)
    if 'age' not in df.columns and 'birth_date' in df.columns:
        df['birth_date'] = pd.to_datetime(df['birth_date'])
        df['age'] = 2025 - df['birth_date'].dt.year
    
    # 2. 부종(Edema) 점수 매핑
    # N(None): 0.0, S(Slight): 0.5, Y(Severe): 1.0
    edema_map = {'N': 0.0, 'S': 0.5, 'Y': 1.0}
    if df['swelling'].dtype == 'object':
        edema_score = df['swelling'].map(edema_map).fillna(0.0)
    else:
        # 이미 숫자로 변환된 경우 (0, 1, 2) -> (0.0, 0.5, 1.0)으로 스케일 조정
        edema_score = df['swelling_ord'] * 0.5 
        
    # 3. Mayo PBC Risk Score 공식 적용
    # R = 0.871*ln(Bilirubin) - 2.53*ln(Albumin) + 0.039*Age + 2.38*ln(Prothrombin) + 0.859*Edema
    # 매핑: Bilirubin -> chem_01, Albumin -> protein_level, Prothrombin -> clot_time
    
    # 로그 변환 시 0 이하 값 방지 (1e-6 더함)
    df['mayo_score'] = (
        0.871 * np.log(df['chem_01'] + 1e-6) 
        - 2.53 * np.log(df['protein_level'] + 1e-6) 
        + 0.039 * df['age'] 
        + 2.38 * np.log(df['clot_time'] + 1e-6) 
        + 0.859 * edema_score
    )
    
    # 4. Mayo Risk Group (공식 논문 기준 Cut-off 적용)
    # 4.5 미만: 저위험(0), 4.5~7.8: 중등도(1), 7.8 이상: 고위험(2)
    df['mayo_risk_group'] = pd.cut(
        df['mayo_score'], 
        bins=[-np.inf, 4.5, 7.8, np.inf], 
        labels=[0, 1, 2]
    ).astype(int)
    
    return df

def add_log_features(df):
    df = df.copy()
    
    # 로그 변환 대상: 범위가 넓고 치우친 변수들
    # 0이나 음수가 있을 수 있으므로 np.log1p (log(x+1)) 사용이 안전함
    target_cols = ['enzyme_A', 'enzyme_B', 'chem_01', 'chem_02', 'trace_metal', 'obs_days']
    
    for col in target_cols:
        if col in df.columns:
            # 음수 값이 있다면 shift 후 로그 변환 (안전장치)
            min_val = df[col].min()
            if min_val < 0:
                df[f'log_{col}'] = np.log1p(df[col] - min_val)
            else:
                df[f'log_{col}'] = np.log1p(df[col])
                
    return df

# --------------------------------------------------------------------------------
# [Part 3] 기존 심화 전처리 함수들 (Caution, Post-EDA, Group Stats, QBins)
# --------------------------------------------------------------------------------

def add_discrete_caution_features(X: pd.DataFrame) -> pd.DataFrame:
    X = X.copy()
    
    X["stage_mid"] = X["disease_stage"].between(2, 3).astype(int)
    X["beh_mid"]   = (X["behavior_index"] == 1).astype(int)

    X["caution_symptom_zone"] = (
        X["symptom_count"].between(1, 2) & (X["swelling_ord"] >= 1)
    ).astype(int)

    X["caution_stage_beh"] = (
        (X["stage_mid"] == 1) & (X["beh_mid"] == 1)
    ).astype(int)

    # organ_enlarge가 Y/N일 수도 있고 0/1일 수도 있음
    if X['organ_enlarge'].dtype == 'object':
        organ_flag = (X['organ_enlarge'] == 'Y')
    else:
        organ_flag = (X['organ_enlarge'] == 1)

    X["caution_organ_only"] = (
        organ_flag
        & (X["disease_stage"] == 1)
        & (X["behavior_index"] <= 1)
    ).astype(int)

    return X

def add_post_eda_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # Treatment Interaction
    df['protein_treatment_interaction'] = df['protein_level'] * (1 + df['treatment'])
    df['blood_treatment_interaction']   = df['blood_cells'] * (1 + df['treatment'])
    df['chem01_treatment_interaction']  = df['chem_01'] * (1 + df['treatment'])
    df['trace_treatment_interaction']   = df['trace_metal'] * (1 + df['treatment'])

    # Coagulation Index
    df['coagulation_index'] = df['protein_level'] / (df['clot_time'] + 1e-6)
    df['coagulation_product'] = df['protein_level'] * df['clot_time']

    # Deadly Combo
    if df['organ_enlarge'].dtype == 'object':
        organ_val = (df['organ_enlarge'] == 'Y').astype(int)
    else:
        organ_val = df['organ_enlarge']
        
    df['deadly_combo'] = df['swelling_ord'] * organ_val * 2
    
    # Obs Days Nonlinearity
    df['log_obs_days'] = np.log1p(df['obs_days'])
    df['obs_days_squared'] = df['obs_days'] ** 2

    return df

def add_group_stats_no_leakage(train: pd.DataFrame, test: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    train = train.copy()
    test = test.copy()
    
    bins = [60, 80, 90, 100, 130]
    labels = ["60-79", "80-89", "90-99", "100+"]
    
    for df in [train, test]:
        df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=False, include_lowest=True)

    group_cols = ['disease_stage', 'gender', 'age_group']
    agg_cols = ['enzyme_A', 'enzyme_B', 'protein_level', 'immune_index', 'lipid_index', 'blood_cells']

    for group_col in group_cols:
        for col in agg_cols:
            train_groupby = train.groupby(group_col)[col]
            mean_map = train_groupby.mean()
            std_map  = train_groupby.std()
            
            prefix = f'{col}_by_{group_col}'
            
            for df_target in [train, test]:
                mapped_mean = df_target[group_col].map(mean_map).astype(float).values
                mapped_std  = df_target[group_col].map(std_map).astype(float).values
                col_values  = df_target[col].astype(float).values
                
                df_target[f'{prefix}_diff_mean']  = col_values - mapped_mean
                df_target[f'{prefix}_zscore']     = (col_values - mapped_mean) / (mapped_std + 1e-6)
                df_target[f'{prefix}_ratio_mean'] = col_values / (mapped_mean + 1e-6)

    return train, test

def add_qbins_no_leakage(train: pd.DataFrame, test: pd.DataFrame, n_bins=4) -> (pd.DataFrame, pd.DataFrame):
    train = train.copy()
    test = test.copy()
    
    lab_cols = [
        "chem_01", "chem_02", "protein_level", "trace_metal",
        "enzyme_A", "enzyme_B", "lipid_index", "blood_cells", "clot_time",
        "mayo_score" # mayo_score도 구간화 대상에 추가 (권장)
    ]
    
    for col in lab_cols:
        if col not in train.columns: continue
        
        try:
            _, bins = pd.qcut(train[col], q=n_bins, retbins=True, duplicates='drop')
        except ValueError:
            continue
            
        train[f"{col}_qbin"] = pd.cut(train[col], bins=bins, labels=False, include_lowest=True).fillna(-1).astype(int)
        test[f"{col}_qbin"] = pd.cut(test[col], bins=bins, labels=False, include_lowest=True).fillna(-1).astype(int)
        
    return train, test


# --------------------------------------------------------------------------------
# [Part 4] 전체 전처리 실행 함수 (통합 파이프라인)
# --------------------------------------------------------------------------------

def run_feature_engineering(train_df, test_df):
    print("1. 기본 전처리 (나이 변환 등)...")
    train_df = prepare_raw_simple(train_df)
    test_df = prepare_raw_simple(test_df)

    print("2. EDA Rule 적용...")
    train_df = add_eda_rule_features_v2(train_df)
    test_df  = add_eda_rule_features_v2(test_df)

    print("3. Boosting Feature 생성...")
    train_df = make_boosting_features_v2(train_df)
    test_df  = make_boosting_features_v2(test_df)
    
    print("⭐ 3-1. Mayo PBC Score & Risk Group 추가 (NEW)...")
    train_df = add_mayo_pbc_features(train_df)
    test_df  = add_mayo_pbc_features(test_df)

    print("3-2. Log Features 생성...")
    train_df = add_log_features(train_df)
    test_df  = add_log_features(test_df)

    print("4. Caution Feature 생성...")
    train_df = add_discrete_caution_features(train_df)
    test_df  = add_discrete_caution_features(test_df)

    print("4-1. EDA 심화 피처(Treatment Interaction 등) 생성...")
    train_df = add_post_eda_features(train_df)
    test_df  = add_post_eda_features(test_df)

    print("5. Group Stats (No Leakage) 생성...")
    train_df, test_df = add_group_stats_no_leakage(train_df, test_df)

    print("6. Q-Binning (No Leakage) 생성...")
    train_df, test_df = add_qbins_no_leakage(train_df, test_df)
    

    

    return train_df, test_df
# 사용 예시:
# train_df, test_df = run_feature_engineering(train_df, test_df)

In [36]:
# %% [Section 2] 데이터 로드 및 전처리 실행

# 1. 데이터 로드 (경로를 본인 환경에 맞게 수정하세요)
train_path = r"C:\Users\abc01\OneDrive\바탕 화면\train.csv"
test_path  = r"C:\Users\abc01\OneDrive\바탕 화면\test.csv"

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

print(f"Original Train shape: {train_df.shape}")
print(f"Original Test shape:  {test_df.shape}")

# 2. 전처리 파이프라인 실행 (Mayo Score 포함)
# 앞서 정의한 run_feature_engineering 함수를 호출합니다.
print("\n🚀 전처리 및 피처 엔지니어링 시작...")
train_df, test_df = run_feature_engineering(train_df, test_df)

# 3. X, y 분리 및 불필요 컬럼 제거
drop_cols = ["index", "name", "birth_date", "geo_code"]
y = train_df["target"].values
X_all = train_df.drop(columns=drop_cols + ["target"], errors='ignore')
X_all_test = test_df.drop(columns=drop_cols, errors='ignore')

print(f"\nPreprocessed X_all shape: {X_all.shape}")
print(f"Preprocessed X_all_test shape: {X_all_test.shape}")

# %% [Section 3] 스마트 스케일링 (Skewness 기반)
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer

# 1. 스케일링 대상 선정: 순수 수치형 데이터만 (범주형, 바이너리, qbin 제외)
numeric_candidates = X_all.select_dtypes(include=['number']).columns

# 스케일링에서 제외할 컬럼들 (이미 범주형이거나 인코딩된 것들)
EXCLUDE_COLS = [
    "gender_int", "fluid_accum", "organ_enlarge", "vascular_marks", 
    "swelling", "treatment", "geo_code", "risk_segment",
    "chem_01_range", "trace_metal_range", "swelling_ord",
    "mayo_risk_group", # ⭐ NEW: 새로 추가된 범주형 피처도 제외
    "target", "index"
]

# _qbin으로 끝나는 컬럼도 제외 (이미 구간화됨)
NUMERIC_COLS = [c for c in numeric_candidates if c not in EXCLUDE_COLS and not c.endswith('_qbin')]

# 2. 스케일링 함수 (왜도에 따라 Scaler 자동 선택)
def apply_skew_scaling(train_df, test_df, cols):
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # Train 기준 Skewness 계산
    skewness = train_df[cols].skew()
    print(f"\nApplying Skew-based Scaling on {len(cols)} columns...")
    
    for col in cols:
        skew = abs(skewness[col])
        
        if skew < 0.5:
            scaler = StandardScaler()
        elif skew < 1.5:
            scaler = RobustScaler()
        else:
            # 왜도가 심하면 정규분포로 강제 변환
            scaler = QuantileTransformer(output_distribution="normal", random_state=42)
            
        train_df[col] = scaler.fit_transform(train_df[[col]]).flatten()
        test_df[col] = scaler.transform(test_df[[col]]).flatten()
            
    return train_df, test_df

# 3. 스케일링 적용
X_all, X_all_test = apply_skew_scaling(X_all, X_all_test, NUMERIC_COLS)
print("✅ Smart Scaling Complete.")

Original Train shape: (6500, 24)
Original Test shape:  (1405, 23)

🚀 전처리 및 피처 엔지니어링 시작...
1. 기본 전처리 (나이 변환 등)...
2. EDA Rule 적용...
3. Boosting Feature 생성...
⭐ 3-1. Mayo PBC Score & Risk Group 추가 (NEW)...
3-2. Log Features 생성...
4. Caution Feature 생성...
4-1. EDA 심화 피처(Treatment Interaction 등) 생성...
5. Group Stats (No Leakage) 생성...
6. Q-Binning (No Leakage) 생성...

Preprocessed X_all shape: (6500, 131)
Preprocessed X_all_test shape: (1405, 131)

Applying Skew-based Scaling on 105 columns...
✅ Smart Scaling Complete.


In [37]:
# %% [Section 4] 스마트 스케일링 (Skewness 기반)

# 1. 스케일링 대상: 순수 수치형 데이터만 (범주형, 바이너리 제외)
numeric_candidates = X_all.select_dtypes(include=['number']).columns

EXCLUDE_COLS = [
    "gender_int", "fluid_accum", "organ_enlarge", "vascular_marks", 
    "swelling", "treatment", "geo_code", "risk_segment",
    "chem_01_range", "trace_metal_range", "swelling_ord",
    "target", "index"
]

NUMERIC_COLS = [c for c in numeric_candidates if c not in EXCLUDE_COLS and not c.endswith('_qbin')]

# 2. 스케일링 함수
def apply_skew_scaling(train_df, test_df, cols):
    train_df = train_df.copy()
    test_df = test_df.copy()
    
    # Train 기준 Skewness 계산
    skewness = train_df[cols].skew()
    print(f"Applying Skew-based Scaling on {len(cols)} columns...")
    
    for col in cols:
        skew = abs(skewness[col])
        
        if skew < 0.5:
            scaler = StandardScaler()
        elif skew < 1.5:
            scaler = RobustScaler()
        else:
            scaler = QuantileTransformer(output_distribution="normal", random_state=42)
            
        train_df[col] = scaler.fit_transform(train_df[[col]]).flatten()
        test_df[col] = scaler.transform(test_df[[col]]).flatten()
            
    return train_df, test_df

# 3. 적용
X_all, X_all_test = apply_skew_scaling(X_all, X_all_test, NUMERIC_COLS)
print("Smart Scaling Complete.")

Applying Skew-based Scaling on 106 columns...
Smart Scaling Complete.


In [55]:
# %% [Final Fix] 정예 36개 + Mayo Score 조합 검증

from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import numpy as np

# 1. 전설의 36개 피처 (0.69955 기록)
CHAMPION_FEATS = [
    'chem_01_range', 'chem01_trace_combo', 'chem_01', 'symptom_count', 'clot_time_qbin', 
    'symptom_severity', 'health_per_stage', 'obs_days', 'swelling_ord', 'gender_int', 
    'age', 'lipid_index_by_gender_ratio_mean', 'chem_01_qbin', 'enzyme_B_by_disease_stage_zscore', 
    'blood_cells_by_gender_zscore', 'enzyme_A_by_gender_diff_mean', 'chem01_treatment_interaction', 
    'deadly_combo', 'enzyme_A_by_disease_stage_ratio_mean', 'enzyme_B', 'sum_enzyme', 
    'chem01_chem02_combo', 'sum_chem', 'blood_cells_by_gender_ratio_mean', 
    'trace_treatment_interaction', 'enzyme_B_by_age_group_ratio_mean', 
    'lipid_index_by_disease_stage_ratio_mean', 'blood_cells_by_age_group_diff_mean', 
    'enzyme_A', 'enzyme_B_by_disease_stage_diff_mean', 'blood_treatment_interaction', 
    'blood_cells_by_disease_stage_diff_mean', 'enzyme_ratio', 'behavior_per_age', 
    'lipid_index_qbin', 'treatment'
]

# 2. 여기에 Mayo만 딱 추가! (나머지 잡다한 건 버림)
# mayo_score, mayo_risk_group, 그리고 혹시 모를 log_mayo 등
MAYO_FEATS = [c for c in X_all.columns if 'mayo' in c] 

FINAL_COMBINATION = list(set(CHAMPION_FEATS + MAYO_FEATS))
FINAL_COMBINATION = [c for c in FINAL_COMBINATION if c in X_all.columns]

print(f"🛡️ 챔피언(36) + ⚔️ Mayo({len(MAYO_FEATS)})")
print(f"🚀 최종 피처 개수: {len(FINAL_COMBINATION)}개")
print(f"   목록: {FINAL_COMBINATION}")

# 3. 데이터셋 준비
X_final = X_all[FINAL_COMBINATION].copy()
y_final = y

# 4. 검증 (파라미터는 기존 최적값 사용)
# Mayo가 들어갔으니 colsample_bytree만 살짝 낮춰서(0.5) Mayo 독재를 막아줍니다.
final_params_xgb = {
    'n_estimators': 788, 
    'learning_rate': 0.06238, 
    'max_depth': 6, 
    'min_child_weight': 1, 
    'gamma': 0.396, 
    'subsample': 0.921, 
    'colsample_bytree': 0.5,  # <--- [수정] Mayo 과적합 방지용 (기존 0.549 -> 0.5)
    'reg_alpha': 0.0027, 
    'reg_lambda': 0.0154, 
    'objective': 'multi:softprob', 
    'random_state': 42, 
    'n_jobs': -1,
    'verbosity': 0
}

print("\n🚀 [Final Check] 5-Fold CV 시작...")
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
fold_scores = []
model = XGBClassifier(**final_params_xgb)

for i, (train_idx, val_idx) in enumerate(kf.split(X_final, y_final), 1):
    X_tr, X_val = X_final.iloc[train_idx], X_final.iloc[val_idx]
    y_tr, y_val = y_final[train_idx], y_final[val_idx]
    
    model.fit(X_tr, y_tr)
    preds = model.predict_proba(X_val)
    
    if preds.shape[1] > 2:
        y_bin = label_binarize(y_val, classes=np.unique(y_final))
        score = average_precision_score(y_bin, preds, average="macro")
    else:
        score = average_precision_score(y_val, preds[:, 1], average="macro")
        
    fold_scores.append(score)
    print(f"   [Fold {i}] mAP: {score:.5f}")

print("\n" + "="*40)
print(f"🏆 최종 평균 mAP: {np.mean(fold_scores):.6f}")
print("="*40)

# 5. 이 점수가 0.699보다 높으면 바로 이걸로 저장
if np.mean(fold_scores) > 0.685: # 아까 0.685보단 무조건 높아야 함
    X_xgb = X_final
    X_xgb_test = X_all_test[FINAL_COMBINATION].copy()
    print("✅ X_xgb 데이터셋이 [챔피언+Mayo] 조합으로 업데이트되었습니다!")

🛡️ 챔피언(36) + ⚔️ Mayo(3)
🚀 최종 피처 개수: 39개
   목록: ['symptom_severity', 'blood_cells_by_age_group_diff_mean', 'gender_int', 'lipid_index_by_gender_ratio_mean', 'chem_01_qbin', 'sum_chem', 'trace_treatment_interaction', 'enzyme_B', 'mayo_score', 'deadly_combo', 'treatment', 'blood_cells_by_disease_stage_diff_mean', 'chem01_treatment_interaction', 'chem01_chem02_combo', 'enzyme_ratio', 'lipid_index_by_disease_stage_ratio_mean', 'mayo_risk_group', 'chem_01', 'enzyme_B_by_disease_stage_diff_mean', 'symptom_count', 'health_per_stage', 'chem_01_range', 'age', 'sum_enzyme', 'enzyme_B_by_age_group_ratio_mean', 'obs_days', 'behavior_per_age', 'enzyme_B_by_disease_stage_zscore', 'lipid_index_qbin', 'blood_cells_by_gender_ratio_mean', 'mayo_score_qbin', 'blood_cells_by_gender_zscore', 'swelling_ord', 'enzyme_A_by_disease_stage_ratio_mean', 'clot_time_qbin', 'chem01_trace_combo', 'enzyme_A', 'enzyme_A_by_gender_diff_mean', 'blood_treatment_interaction']

🚀 [Final Check] 5-Fold CV 시작...
   [Fold 1] mAP

In [56]:
# %% [Fine Tuning] 샘플링 비율 정밀 튜닝 (황금 비율 찾기)

import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import numpy as np

# 1. 데이터 준비 (아까 성공한 X_final 그대로 사용)
# 만약 변수가 날아갔다면 다시 정의하세요.
# X_final = ... 
# y = ...

print(f"🎯 현재 최고 기록에 도전합니다. (Feature 수: {X_final.shape[1]})")

def score_function(y_true, y_pred_proba):
    if y_pred_proba.shape[1] > 2:
        y_bin = label_binarize(y_true, classes=np.unique(y_true))
        return average_precision_score(y_bin, y_pred_proba, average="macro")
    else:
        return average_precision_score(y_true, y_pred_proba[:, 1], average="macro")

def objective_sampling(trial):
    # 1. 샘플링 파라미터 집중 탐색
    # 0.3 (엄격한 견제) ~ 0.8 (약한 견제) 사이를 샅샅이 뒤집니다.
    colsamp = trial.suggest_float('colsample_bytree', 0.3, 0.8)
    subsamp = trial.suggest_float('subsample', 0.6, 1.0)
    
    # 2. 나머지 파라미터는 '성공했던 값'으로 고정 (변수 통제)
    # (아까 0.704 찍었던 그 설정입니다)
    param = {
        'n_estimators': 1000,
        'learning_rate': 0.06238,
        'max_depth': 6,
        'min_child_weight': 1,
        'gamma': 0.396,
        'reg_alpha': 0.0027,
        'reg_lambda': 0.0154,
        
        # 튜닝 대상
        'colsample_bytree': colsamp,
        'subsample': subsamp,
        
        'objective': 'multi:softprob',
        'random_state': 42,
        'n_jobs': -1,
        'verbosity': 0
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in skf.split(X_final, y):
        X_tr, X_val = X_final.iloc[train_idx], X_final.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        
        model = XGBClassifier(**param)
        model.fit(X_tr, y_tr)
        preds = model.predict_proba(X_val)
        scores.append(score_function(y_val, preds))

    return np.mean(scores)

print("\n🚀 샘플링 비율 정밀 튜닝 시작 (30회)...")
study_samp = optuna.create_study(direction='maximize')
study_samp.optimize(objective_sampling, n_trials=30, show_progress_bar=True)

print(f"\n🏆 Best mAP: {study_samp.best_value:.6f}")
print("✨ 찾은 황금 비율:")
print(f"   - colsample_bytree (피처 랜덤): {study_samp.best_params['colsample_bytree']:.4f}")
print(f"   - subsample (데이터 랜덤): {study_samp.best_params['subsample']:.4f}")

[I 2025-11-25 19:48:54,677] A new study created in memory with name: no-name-af1e6cd1-a561-4705-962e-8b989f9f5f23


🎯 현재 최고 기록에 도전합니다. (Feature 수: 39)

🚀 샘플링 비율 정밀 튜닝 시작 (30회)...


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-11-25 19:49:03,395] Trial 0 finished with value: 0.6951047153226337 and parameters: {'colsample_bytree': 0.7928885003809546, 'subsample': 0.864091722324958}. Best is trial 0 with value: 0.6951047153226337.
[I 2025-11-25 19:49:15,420] Trial 1 finished with value: 0.6926670836357881 and parameters: {'colsample_bytree': 0.7436166029602881, 'subsample': 0.6475739139275513}. Best is trial 0 with value: 0.6951047153226337.
[I 2025-11-25 19:49:27,599] Trial 2 finished with value: 0.6940191127540823 and parameters: {'colsample_bytree': 0.6358989262830876, 'subsample': 0.751328638433239}. Best is trial 0 with value: 0.6951047153226337.
[I 2025-11-25 19:49:43,570] Trial 3 finished with value: 0.6943264068422199 and parameters: {'colsample_bytree': 0.7653239810592734, 'subsample': 0.6933427342615919}. Best is trial 0 with value: 0.6951047153226337.
[I 2025-11-25 19:49:58,504] Trial 4 finished with value: 0.7002991388935882 and parameters: {'colsample_bytree': 0.3182957983107913, 'subsampl

In [63]:
# %% [LGBM Fine Tuning] 기존 파라미터 고정 + 샘플링 비율만 집중 튜닝

import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import label_binarize
import numpy as np

# 1. 기존에 찾은 꿀 파라미터 (고정값)
base_params_lgb = {
    'n_estimators': 912,
    'learning_rate': 0.01090933368427726,
    'num_leaves': 193,
    'max_depth': 14,
    'min_child_samples': 51,
    # 'subsample': 0.5933...  <-- 이건 튜닝할 거니까 제외
    # 'colsample_bytree': 0.5036... <-- 이것도 튜닝 대상
    'reg_alpha': 0.008835424632453443,
    'reg_lambda': 0.07034822683895961,
    'objective': 'multiclass',
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}

# 2. 데이터 준비 (Mayo가 포함된 X_lgb_final 사용)
# X_lgb_final = ... (이전 단계에서 만들어진 데이터셋)
# y = ...

print(f"🎯 LGBM 샘플링 최적화 시작 (Base Feature 수: {X_lgb_final.shape[1]})")

def score_function(y_true, y_pred_proba):
    if y_pred_proba.shape[1] > 2:
        y_bin = label_binarize(y_true, classes=np.unique(y_true))
        return average_precision_score(y_bin, y_pred_proba, average="macro")
    else:
        return average_precision_score(y_true, y_pred_proba[:, 1], average="macro")

def objective_lgb_sampling(trial):
    # 튜닝 대상: 샘플링 파라미터만 (0.3 ~ 0.8 범위 집중 탐색)
    # Mayo Score가 강력하므로, 피처를 적게 뽑는(0.3~0.5) 쪽이 유리할 수 있음
    colsamp = trial.suggest_float('colsample_bytree', 0.3, 0.8)
    subsamp = trial.suggest_float('subsample', 0.5, 1.0)
    
    # 파라미터 합치기
    param = base_params_lgb.copy()
    param['colsample_bytree'] = colsamp
    param['subsample'] = subsamp
    param['subsample_freq'] = 1 # 배깅 활성화를 위해 필수
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in skf.split(X_lgb_final, y):
        X_tr, X_val = X_lgb_final.iloc[train_idx], X_lgb_final.iloc[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]
        
        model = LGBMClassifier(**param)
        model.fit(X_tr, y_tr)
        preds = model.predict_proba(X_val)
        scores.append(score_function(y_val, preds))

    return np.mean(scores)

print("\n🚀 LGBM 샘플링 비율 튜닝 시작 (30회)...")
study_lgb_samp = optuna.create_study(direction='maximize')
study_lgb_samp.optimize(objective_lgb_sampling, n_trials=30, show_progress_bar=True)

print(f"\n🏆 Best LGBM mAP: {study_lgb_samp.best_value:.6f}")
print("✨ 찾은 황금 비율:")
print(f"   - colsample_bytree: {study_lgb_samp.best_params['colsample_bytree']:.4f}")
print(f"   - subsample: {study_lgb_samp.best_params['subsample']:.4f}")

# 3. 최종 파라미터 확정
final_params_lgb_tuned = base_params_lgb.copy()
final_params_lgb_tuned.update(study_lgb_samp.best_params)
final_params_lgb_tuned['subsample_freq'] = 1

print("\n✅ 최종 파라미터 준비 완료 (final_params_lgb_tuned)")

[I 2025-11-26 06:40:49,163] A new study created in memory with name: no-name-83a86386-1a3d-4a06-a125-f1943390e38e


🎯 LGBM 샘플링 최적화 시작 (Base Feature 수: 22)

🚀 LGBM 샘플링 비율 튜닝 시작 (30회)...


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-11-26 06:41:12,535] Trial 0 finished with value: 0.6941275694802227 and parameters: {'colsample_bytree': 0.6848904570319175, 'subsample': 0.5232583014550416}. Best is trial 0 with value: 0.6941275694802227.
[I 2025-11-26 06:41:41,508] Trial 1 finished with value: 0.6966609679283466 and parameters: {'colsample_bytree': 0.5915280229860977, 'subsample': 0.9267769135624113}. Best is trial 1 with value: 0.6966609679283466.
[I 2025-11-26 06:42:10,999] Trial 2 finished with value: 0.6967511511723345 and parameters: {'colsample_bytree': 0.745574712812131, 'subsample': 0.7560804742359675}. Best is trial 2 with value: 0.6967511511723345.
[I 2025-11-26 06:42:30,277] Trial 3 finished with value: 0.6979561380544173 and parameters: {'colsample_bytree': 0.7573334514813482, 'subsample': 0.5721253088710312}. Best is trial 3 with value: 0.6979561380544173.
[I 2025-11-26 06:42:51,828] Trial 4 finished with value: 0.69893184684255 and parameters: {'colsample_bytree': 0.34640559220178857, 'subsampl